#### Jenish Dobariya

#### Date: 9th Febuary 2024

### Loading inverted index (H) and computing DocLen (DL).

In [1]:
import csv
import math

tindexfile = 'medline_term_index.csv'
invindexfile = 'medline_inverted_index.csv'
dindexfile = 'medline_doc_index.csv'

# Number of documents in the corpus (hard-coded for this corpus)
N = 1033

# Major data structures
H_invindex = {} # inverted index; term -> (idf, L:hashmap of (docID . tf))
DL_doclen = {}  # document lengths; docID -> len

## (1) Read the term index file and populate the invindex first
tid2term_map = {} # temporary storage to hold mappings of termID -> term

fin = open(tindexfile, 'r', encoding='utf-8')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    term = line[0]    # term string
    termID = line[1]  # termID
    df = int(line[2]) # document frequency
    idf = math.log10(N/df) # idf
    # record term -> (idf, emptyL) in H
    H_invindex[term] = (idf, dict())
    # record termID -> term 
    tid2term_map[termID] = term 
fin.close()

## (2) Read the inverted index file and add postings lists in H.
## Also compute document lengths too, incrementally -- and record in DL.
fin = open(invindexfile, 'r')
reader = csv.reader(fin, delimiter='\t')
for line in reader:
    termID = line[0]
    idx = 1
    while idx < (len(line)-1):
        docID = line[idx]
        tf = int(line[idx+1]) # raw tf of the term in this document
        # Record docID -> tf in term's L
        L = (H_invindex[tid2term_map[termID]])[1]
        L[docID] = tf  # docID -> raw term frequency
        
        # Accumulate the component vector length for the document
        tfidf = tf * (H_invindex[tid2term_map[termID]])[0] # tf * idf
        tfidfsq = math.pow(tfidf, 2.0)
        if docID in DL_doclen:
            DL_doclen[docID] += tfidfsq
        else:
            DL_doclen[docID] = tfidfsq
        #
        idx += 2
fin.close()

# Fix the DL entries by applying sqrt to make vector length.
for docID in DL_doclen.keys():
    val = DL_doclen[docID]
    DL_doclen[docID] = math.sqrt(val)

    
print ('Total # terms: %d' % len(H_invindex))
for term in ['pentobarbit', 'defici', 'treatment']:
    print (' - Entry for \'%s\': df=%s, idf=%s' % (term, len(H_invindex[term][1]), H_invindex[term][0]))

print ('\nTotal # documents: %d' % len(DL_doclen))
for docID in ['59', '1033']:
    print (' - Vector len for Doc %s = %s' % (docID, DL_doclen[docID]))


Total # terms: 11463
 - Entry for 'pentobarbit': df=4, idf=2.412040330191658
 - Entry for 'defici': df=39, idf=1.4230357144931214
 - Entry for 'treatment': df=172, idf=0.7785718746120717

Total # documents: 1033
 - Vector len for Doc 59 = 13.811725366348801
 - Vector len for Doc 1033 = 31.163653356034512


### Queries as Vectors

In [2]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

queryfile = 'medline.query'

# A list of queries. Each Query is a tuple, (qID, Q:term->tf map)
Queries_list = []

fin = open(queryfile, 'r', encoding='utf-8')#'iso-8859-1')
porter = nltk.PorterStemmer()

for line in fin:
    matchObj = re.match(r'^(\d+)\s+(.*)', line)
    if not matchObj:
        print ("ERROR with line -- %s" % line)
    else:
        queryID = matchObj.group(1) # queryID
        text = matchObj.group(2)    # query string (ignoring sentences)

        # process text string -- same processing as one applied to documents.
        tokens = word_tokenize(text.lower())
        terms = [porter.stem(w) for w in tokens if w not in stopwords.words('english') and len(w) > 1] # (**)
        # term frequencies of the terms in this query are obtained by NLTK's FreqDist
        fdist = nltk.FreqDist(terms)
        # append the Query in the list
        Queries_list.append((queryID, dict(fdist)))
fin.close()

print ('Total # queries: %d' % len(Queries_list))
for qid in [1, 21]:
    print (' - Query %s: %s' % (Queries_list[qid][0], Queries_list[qid][1]))

Total # queries: 30
 - Query 2: {'relationship': 1, 'blood': 1, 'cerebrospin': 1, 'fluid': 1, 'oxygen': 1, 'concentr': 1, 'partial': 1, 'pressur': 1, 'method': 1, 'interest': 1, 'polarographi': 1}
 - Query 22: {'mycoplasma': 1, 'infect': 1, 'presenc': 1, 'embryo': 1, 'fetu': 1, 'newborn': 1, 'infant': 1, 'anim': 1, 'pregnanc': 1, 'gynecolog': 1, 'diseas': 1, 'relat': 1, 'chromosom': 2, 'abnorm': 1}


## Retrieval and Ranking

In [3]:
# Importing numpy for computational purposes
import numpy as np

In [4]:
def compute_query_vector(query_terms, H_invindex):
    # Getting a query vector the size of H_invindex
    query_vector = np.zeros(len(H_invindex))
    
    # Iterating through the terms and tuples in inverted index
    for term_idx, (term, (idf,_)) in enumerate(H_invindex.items()):
        if term in query_terms:
            tf = query_terms[term] # Creating TF value
            
             # getting the query vector value with tf_iDF
            query_vector[term_idx] = tf * idf
    return query_vector

In [5]:
# Initiating a list to store all query vector
query_vectors = []

# Iterating though the Queries_list created before 
for query_id, query_terms in Queries_list:
    # Passing the query_term and inverted index to get the query_vector
    query_vector = compute_query_vector(query_terms, H_invindex)
    
    # storing the new found vector to the list
    query_vectors.append((query_id, query_vector))

In [6]:
# Initializing an empty dictionary to store document vectors
document_vectors = {}

# Iterating through DL_doclen
for doc_id, doc_vector in DL_doclen.items():
    # Initializing zero vectors as the length of inverted index
    document_vector = np.zeros(len(H_invindex))
    
    # Iterating though inverted index
    for term_idx, (term, (idf, _)) in enumerate(H_invindex.items()):
        if doc_id in H_invindex[term][1]:
            # getting tf value
            tf = H_invindex[term][1][doc_id]
            
            # calculating the document vector
            document_vector[term_idx] = tf * idf
    document_vectors[doc_id] = document_vector

In [7]:
def cosine_similarity(query_vector, doc_vector):
    # Computing the dot product using numpy between query and doc vector
    dot_product = np.dot(query_vector, doc_vector)
    # Computing L2 norm of the query and document vector
    query_norm = np.linalg.norm(query_vector)
    doc_norm = np.linalg.norm(doc_vector)
    
    # If both are non zero compute the cosine similarity 
    if query_norm != 0 and doc_norm != 0:
        return dot_product / (query_norm * doc_norm)
    else:
        return 0

In [8]:
# Initializing empty list to store the ranked list
ranked_lists = []

# Itereating through the query vectors
for query_id, query_vector in query_vectors:
    similarities = [] # Initializing an empty list to store the document similarities
    
    # Iterating through the document vectors to find cosine similarity
    for doc_id, doc_vector in document_vectors.items():
        if cosine_similarity(query_vector, doc_vector) != 0:
            similarities.append((doc_id, cosine_similarity(query_vector, doc_vector)))
    
    # Sorting the list of similarities based on similarity scores in decending order
    ranked_list = sorted(similarities, key=lambda x: x[1], reverse=True)
    ranked_lists.append((query_id, ranked_list))

In [9]:
# Initializing an empty dictionary to store query results
query_results = {}

# Iterating through ranked_lists
for query_ids, ranked_list in ranked_lists:
    ranked_document_ids = [] # Initializing an empty list to store docIDs
    
    # Iterating through ranked doc in the ranked list
    for rank, (doc_ids, similarity) in enumerate(ranked_list):
        ranked_document_ids.append(doc_ids)
        
    # Store the list of ranked document ids in the query results dictionary
    query_results[query_ids] = ranked_document_ids

In [10]:
# Writing to a file
with open("rankedlist.txt", "w") as fin:
    # Iterating through query_results dictionary
    for query_id, document_ids in query_results.items():
        # Joining all doc_ids separated by comma
        document_ids_line = ", ".join(document_ids)
        # Writing to the file
        fin.write(f"{query_id}, {document_ids_line}\n")
        
fin.close() # Closing the file

## Evaluation -- Computing Mean Average Precision (MAP)

In [11]:
def read_answers_file(file_path):
    answers = {} # Creating an empty dictionary to store doc_id and key values
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(" ") # Separating all values in a line
            query_id = parts[0] # Getting queryid at index 0
            relevant_docs = parts[1:] # getting relevent docids
            
            # Appending the queryid and docids to answer
            answers[query_id] = relevant_docs
    return answers

In [12]:
answer = read_answers_file('medline.rel')

# storing the queryid and docids provided by the algorithm
my_res = {}

# Iterating through Ranked_list to get queryid and docid
for query_id, ranked_list in ranked_lists:
    my_docs = [doc_ids for doc_ids, _ in ranked_list]
    my_res[query_id] = my_docs

In [13]:
def calculate_avg_precision(ranked_dic, eval_dic):
        average_precision_dic = {} # Creating empty dictionary
        # Iterating through the ranked_dict
        for q,dID in ranked_dic.items():
            precisionk = 0.0
            relevant_cnt = 0
            # Iterating through the document ID from ranked_dictionary
            for k, docID in enumerate(dID, start=1):
                
                # Checking if the docID is in the evaluation dictionary
                if docID in eval_dic[q]:
                    relevant_cnt +=1
                    precisionk += relevant_cnt / k
            
            # Calculating average precision
            average_precision = precisionk / relevant_cnt
            average_precision_dic[q] = average_precision
            
        return average_precision_dic

In [14]:
out = calculate_avg_precision(my_res, answer)

In [15]:
MAP = 0.0

# Iterating through the dictionary to sum the average precision
for i,j in out.items():
    MAP+=j

# Getting the Mean Average Precision
print(f"MAP: {MAP/30}")

MAP: 0.5656605218653455
